In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_dir = '/kaggle/input/tabular-playground-series-nov-2021/train.csv'
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv', index_col = 'id')
df = pd.read_csv(train_dir)
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sort_values # all not null

In [ ]:
df['target'].value_counts() # that's why I use binary classification

In [ ]:
col = df.columns[1:-1]
col

In [ ]:
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve,RocCurveDisplay,ConfusionMatrixDisplay,confusion_matrix,roc_auc_score,accuracy_score
from sklearn import metrics

 # 정규분포로 표준화
scaler = StandardScaler()
df[col] = scaler.fit_transform(df[col])
test[col] = scaler.transform(test[col])

# cross validation - StratifiedKFold
#Takes group information into account to avoid building folds with imbalanced class distributions
#(for binary or multiclass classification tasks).
skf = StratifiedKFold(n_splits=8, random_state=48, shuffle=True)
auc, acc = [], []
preds = np.zeros(test.shape[0])
n = 0

for trn_idx, val_idx in skf.split(df[col], df['target']):
    X_train, X_val = df[col].iloc[trn_idx], df[col].iloc[val_idx]
    y_train, y_val =  df['target'].iloc[trn_idx], df['target'].iloc[val_idx]
    
    model = XGBClassifier(objective='binary:logistic', # 0 or 1
                               learning_rate=0.75,
                               n_jobs=2, 
                               booster='gblinear',
                               updater='coord_descent') 
    model.fit(X_train, y_train ,
              early_stopping_rounds=5,
              eval_set=[(X_val, y_val)]
            ) 

    preds += model.predict_proba(test[col])[:,1] / skf.n_splits
    
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))
    acc.append(accuracy_score(y_val, model.predict(X_val)))
    
    print(f"fold: {n+1} , accuracy: {round(acc[n]*100,3)} , auc: {round(auc[n]*100,3)}")
    n+=1
print(f"the mean AUC is : {round(np.mean(auc)*100,2)} while the mean Accuracy is : {round(np.mean(acc)*100,2)} ")


In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/sample_submission.csv')
sub['target']=preds
sub.to_csv('submission.csv', index=False)

sub